<a href="https://colab.research.google.com/github/GrigoryevV/HF/blob/main/Implementing_a_Retrieval_Augmented_Generation_(RAG)_System_with_OpenAI's%C2%A0API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken

In [ ]:
import os

# Prompt the user for their OpenAI API key
api_key = input("Please enter your OpenAI API key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")


Please enter your OpenAI API key: sss
OPENAI_API_KEY has been set!


In [ ]:
llm_model = "gpt-3.5-turbo"

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
txt_file_path = '/content/drive/MyDrive/ScaleX Innovation/Tutorials and Notebooks/scalexi.txt'
loader = TextLoader(file_path=txt_file_path, encoding="utf-8")
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [ ]:
#data

In [ ]:
data

[Document(page_content="ScaleX Innovation Bio\n\nScaleX Innovation is a pioneering leader in the realm of Generative AI and Large Language Models. With a keen focus on integrating these transformative technologies into business strategies, the company has carved a niche for itself by offering tailored solutions that enhance innovation and operational efficiency. ScaleX's proficiency extends across multiple industry verticals, ensuring that businesses can harness the power of AI-driven digital transformation. Whether it's automating workflows, content analysis, or custom model implementations, ScaleX Innovation stands at the cutting-edge, committed to bridging the gap between technology and business. Their dedication to ethical compliance and versatility makes them a trusted partner for businesses worldwide.س", metadata={'source': '/content/drive/MyDrive/ScaleX Innovation/Tutorials and Notebooks/scalexi.txt'}),
 Document(page_content="Unlock the Potential of Generative AI\nTransform Bus

In [ ]:
# Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [ ]:
# Create conversation chain
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

In [ ]:
query = "What is ScaleX Innovation?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'ScaleX Innovation is a leading company in the field of Generative AI and Large Language Models. They specialize in integrating these technologies into business strategies, offering tailored solutions that boost innovation and operational efficiency. Their services include automating workflows, content analysis, and implementing custom models. They operate across multiple industry verticals, aiming to help businesses harness the power of AI-driven digital transformation. They are committed to ethical compliance and versatility, making them a trusted business partner worldwide.'

In [ ]:
query = "What is the contact information?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'The contact information for ScaleX Innovation is as follows:\n\nAddress: Route Mahdia km 0.5, Pavillon d’Or Building, 3000 Sfax, Tunisia.\n\nEmail: info@scalexi.ai\n\nPhone Number: +216-55-770-606\n\nThey can also be reached out on WhatsApp at the same number.'

In [ ]:
query = "What are the main activities of ScaleX Innovation. Write is as three bullet points."
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'- Specializing in Generative AI and Large Language Models, offering bespoke solutions that drive innovation and automate workflows.\n- Providing cross-domain consultation and business automation with a client-centric approach.\n- Implementing custom Large Language Models, AI-enabled content and data analysis across multiple industry verticals.'